In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install konlpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 57.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 27.0 MB/s eta 0:00:00


In [3]:
import pandas as pd
from konlpy.tag import Mecab
from gensim.models.doc2vec import TaggedDocument
from tqdm import tqdm

In [4]:
df = pd.read_csv('./Naver_cleansed_8S.csv',  sep=',')
df = df.dropna()
df

,platform_id,category,publisher,publication_date,title,content,author,article_url,update_date
0,네이버,금융,부산일보,2024.08.01,"[속보] 몬테네그로 항소법원 “권도형, 한국으로 송환해야”",2022년 국내 투자자 20만 명이 3000억 원에 달하는 피해을 입은 테라루나 사...,이정훈,https://n.news.naver.com/mnews/article/082/000...,수정 날짜 없음
1,네이버,금융,뉴스1,2024.08.01,"WB '중진국 함정' 극복 대표 사례로 韓 꼽아…""개도국의 필독서""",ⓒ News1 정윤미 기자 전민 기자 세계은행은 1일 발한 세계개발보고서에서 중진국...,전민 기자,https://n.news.naver.com/mnews/article/421/000...,2024.08.01
2,네이버,금융,파이낸셜뉴스,2024.08.01,"지난달 5대 은행 가계대출, 7조 넘게 늘어",7월말 기준 715조7383억거래회복에 대출막차 수요 겹쳐3년 3개월만에 최대폭 증...,박문수,https://n.news.naver.com/mnews/article/014/000...,수정 날짜 없음
3,네이버,금융,매일경제,2024.08.01,"“여보, 달러 팔고 엔화 사자”...두달만에 1360원대, 엔화는 920원 눈앞",원화 달러 약세엔화 강세 연동해 일시적 13401350원대 가능성추세적 반등으로 보...,임영신,https://n.news.naver.com/mnews/article/009/000...,수정 날짜 없음
4,네이버,금융,한겨레,2024.08.01,"일본은행 ‘깜짝’ 금리 인상, 엔화도 깜짝 상승",1일 원엔 환율 91071원지난 26일 서울 중구 하나은행 위변조대응센터에서 직원이...,남지현,https://n.news.naver.com/mnews/article/028/000...,2024.08.01
...,...,...,...,...,...,...,...,...,...
105634,네이버,경제일반,국민일보,2024.08.31,가명에 영어까지 ‘꼼수’…반도체 업계 황당 인재 경쟁,반도체 업계 인재 영입 경쟁 치열경업금지서약서 써도 무용지물법적 효력 미미 형식적 ...,나경연,https://n.news.naver.com/mnews/article/005/000...,수정 날짜 없음
105635,네이버,경제일반,국민일보,2024.08.31,한남동에도 무신사… ‘서울 패션 5대 성지’ 입성 완료,30일 공식 오픈하는 무신사 스탠다드 한남 플래그십 스토어 매장 1층 내부 모습 무...,이다연,https://n.news.naver.com/mnews/article/005/000...,수정 날짜 없음
105636,네이버,경제일반,국민일보,2024.08.31,"엔비디아 독주 막겠다…네이버, 연내 ‘한국판 쿠다’ 공개",인텔 가우디 기반 SW 플랫폼 구축 예정완성된 SW 하이퍼클로바X에도 활용 계획생태...,나경연,https://n.news.naver.com/mnews/article/005/000...,수정 날짜 없음
105637,네이버,경제일반,중앙SUNDAY,2024.08.31,"""향료·색소 첨가해도 막걸리""…전통 훼손인가 규제 완화인가",양유정 기자 문제는 이렇게 줄어든 세금 혜택이 유사 막걸리 수십 종을 생산하는 몇몇...,오유진,https://n.news.naver.com/mnews/article/353/000...,2024.08.31


In [5]:
df['title']

,title
0,"[속보] 몬테네그로 항소법원 “권도형, 한국으로 송환해야”"
1,"WB '중진국 함정' 극복 대표 사례로 韓 꼽아…""개도국의 필독서"""
2,"지난달 5대 은행 가계대출, 7조 넘게 늘어"
3,"“여보, 달러 팔고 엔화 사자”...두달만에 1360원대, 엔화는 920원 눈앞"
4,"일본은행 ‘깜짝’ 금리 인상, 엔화도 깜짝 상승"
...,...
105634,가명에 영어까지 ‘꼼수’…반도체 업계 황당 인재 경쟁
105635,한남동에도 무신사… ‘서울 패션 5대 성지’ 입성 완료
105636,"엔비디아 독주 막겠다…네이버, 연내 ‘한국판 쿠다’ 공개"
105637,"""향료·색소 첨가해도 막걸리""…전통 훼손인가 규제 완화인가"


In [6]:
df['article_url']

,article_url
0,https://n.news.naver.com/mnews/article/082/000...
1,https://n.news.naver.com/mnews/article/421/000...
2,https://n.news.naver.com/mnews/article/014/000...
3,https://n.news.naver.com/mnews/article/009/000...
4,https://n.news.naver.com/mnews/article/028/000...
...,...
105634,https://n.news.naver.com/mnews/article/005/000...
105635,https://n.news.naver.com/mnews/article/005/000...
105636,https://n.news.naver.com/mnews/article/005/000...
105637,https://n.news.naver.com/mnews/article/353/000...


In [6]:
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git

In [7]:
cd Mecab-ko-for-Google-Colab

In [8]:
!bash install_mecab-ko_on_colab_light_220429.sh
# 이부분은 업데이트 될 수 있기에 제작자의 github에서 확인하는 것이 좋다.

In [27]:
mecab = Mecab()

tagged_corpus_list = []

# for index, row in tqdm(df.iterrows(), total=len(df)):
#   text = row['content']
#   tag = row['title']
#   tagged_corpus_list.append(TaggedDocument(tags=[tag], words=mecab.morphs(text)))

# print('문서의 수 :', len(tagged_corpus_list))
# 데이터 샘플링
sample_df = df.sample(n=50000)  # 예를 들어, 50,000개 샘플로 학습
sample_tagged_corpus_list = [TaggedDocument(tags=[row['article_url']], words=mecab.morphs(row['content'])) for index, row in sample_df.iterrows()]
print('문서의 수 :', len(sample_tagged_corpus_list))

문서의 수 : 50000


In [28]:
# tagged_corpus_list[0]
sample_tagged_corpus_list[0]

TaggedDocument(words=['중소', '벤처', '기업', '부', '는', '29', '일', '베트남', '하노이', 'L', '7', '호텔', '에서', '9', '월', '동행', '축제', '개막식', '을', '갖', '고', '한', '달', '간', '의', '행사', '에', '들어갔', '다', '이번', '으로', '5', '년', '째', '를', '맞', '는', '동행', '축제', '가', '해외', '에서', '개최', '된', '것', '은', '이번', '이', '처음', '이', '다', '베트남', '은', '지난해', '양국', '간', '교역', '액', '이', '794', '억', '달러', '를', '넘어선', '한국', '의', '3', '대', '무역국', '이', '다', '특히', '한국', '중소기업', '의', '진출', '이', '활발', '하', '며', '뷰티', '푸드', '등', '한국', '제품', '에', '대한', '인지도', '가', '높', '아', '판로', '를', '확대', '할', '수', '있', '는', '잠재력', '이', '매우', '큰', '국가', '로', '국내', '중소기업', '의', '우수', '제품', '을', '해외', '에', '알리', '는', '첫', '출발지', '로', '적합', '하', '다는', '것', '이', '중기', '부', '의', '설명', '이', '다', '오영주', '중소', '벤처', '기업', '부', '장관', '이', '28', '일', '하노이', '베트남', '정부', '영빈관', '에서', '응', '우', '엔', '밍', '부', '베트남', '외교부', '차관', '을', '만나', '양국', '의', '혁신', '중소', '벤처', '기업', '과', '함께', '교류', '협력', '방', '안', '에', '대해', '논의', '하', '고', '있', '다', '중소', '벤처', '기업', 

In [11]:
from gensim.models import doc2vec

In [29]:
# model = doc2vec.Doc2Vec(vector_size=300, alpha=0.025, min_alpha=0.025, workers=8, window=8)

# # Vocabulary 빌드
# model.build_vocab(tagged_corpus_list)

# # Doc2Vec 학습
# model.train(tagged_corpus_list, total_examples=model.corpus_count, epochs=20)

# # 모델 저장
# model.save('dart.doc2vec')
model = doc2vec.Doc2Vec(vector_size=300, alpha=0.025, min_alpha=0.025, workers=8, window=8)
model.build_vocab(sample_tagged_corpus_list)
model.train(sample_tagged_corpus_list, total_examples=model.corpus_count, epochs=10)  # 에폭 수 조정

In [31]:
# 모델 저장
model.save('dart.doc2vec')

# 모델 로드
model = doc2vec.Doc2Vec.load('dart.doc2vec')

In [32]:
similar_doc = model.dv.most_similar('https://n.news.naver.com/mnews/article/022/0003964231')
print(similar_doc)

[('https://n.news.naver.com/mnews/article/082/0001286237', 0.7658616900444031), ('https://n.news.naver.com/mnews/article/001/0014900217', 0.7572737336158752), ('https://n.news.naver.com/mnews/article/029/0002898771', 0.7291800379753113), ('https://n.news.naver.com/mnews/article/421/0007757490', 0.7028075456619263), ('https://n.news.naver.com/mnews/article/421/0007757603', 0.6899970769882202), ('https://n.news.naver.com/mnews/article/021/0002657132', 0.684211015701294), ('https://n.news.naver.com/mnews/article/366/0001014610', 0.6735967397689819), ('https://n.news.naver.com/mnews/article/079/0003931182', 0.6592068076133728), ('https://n.news.naver.com/mnews/article/001/0014901676', 0.640476644039154), ('https://n.news.naver.com/mnews/article/421/0007759601', 0.6391329169273376)]


In [16]:
# similar_doc = model.dv.most_similar('농심, 수출 전용 라면 공장 부산 녹산공단에 짓는다')
# print(similar_doc)

KeyError: "Key '농심, 수출 전용 라면 공장 부산 녹산공단에 짓는다' not present in vocabulary"

In [17]:
similar_doc = model.dv.most_similar('美, 자국 빅테크에 과세하려는 캐나다에 분쟁해결 협의 요청')
print(similar_doc)

[('"EU, 디지털시장법에 기업 생산성 약화…韓 적용시 막대한 피해"', 0.4649980664253235), ('디아이 "종속회사, SK하이닉스에 166억 검사장비 공급"', 0.459530234336853), ("국민연금, 핀란드 배당원천세 환급 '승소'", 0.45283734798431396), ("당근-KAIT, '건전한 디지털 생태계 조성' MOU 체결", 0.4469987452030182), ('[fn마켓워치]국민연금, 핀란드 국세청에 승소', 0.44411247968673706), ('"적대국 거래 제한" 美생물보안법 입법 가시화[제약·바이오 해외토픽]', 0.4415096342563629), ('iM증권, 통합증거금 서비스 이벤트…환전우대쿠폰 제공', 0.4402730166912079), ('중국, 캐나다 中전기차 100% 관세 예고에 "필요한 모든 조치"', 0.4370119571685791), ('iM증권, 통합증거금 서비스 이벤트…97% 환전우대쿠폰 지급', 0.4340507984161377), ('불확실성 커진 미 대선…尹 정부 대응은?', 0.4284374415874481)]


In [19]:
# 유사한 문서 찾기
similar_doc = model.dv.most_similar('"광복 영화" 말하니 목록이 좌르륵…한층 똑똑해진 삼성 AI TV')

# 유사한 문서의 정보를 담을 리스트
similar_articles = []

# 유사한 문서의 제목을 기반으로 원래 데이터프레임에서 정보 가져오기
for title, similarity in similar_doc:
    # 제목을 기준으로 데이터프레임에서 해당 문서 정보 가져오기
    article_info = df[df['title'] == title].iloc[0]

    content = article_info['content']
    news_agency = article_info['publisher']
    url = article_info['article_url']

    # 정보 추가
    similar_articles.append({
        'title': title,
        'content': content,
        'news_agency': news_agency,
        'url': url,
        'similarity': similarity
    })

# 유사한 기사 정보 출력
for article in similar_articles:
    print(f"Title: {article['title']}")
    print(f"Content: {article['content']}")
    print(f"publisher: {article['news_agency']}")
    print(f"URL: {article['url']}")
    print(f"Similarity: {article['similarity']}\n")

Title: "타이젠OS 7년간 무상 업그레이드"…삼성, AI홈 진화 이끈다
Content: AI 스크린으로 AI 홈라이프 경험 개선타이젠OS 업그레이드…대중화 이끈다화질사운드자막 등 AI 맞춤형 시청환경 형사들이 치킨 파는 영화 찾아줘 첫 번째 영화 선택하고 음량 15로 맞춰줘 삼성전자가 최초로 TV에 자연어 기반 맥락을 이해하고 한 번에 두 가지 지시를 명령할 수 있는 인공지능 음성기술을 AI TV에 실현했다 삼성전자는 22일 수원사업장 디지털연구소에서 AI 홈 라이프를 중심으로 진화하는 새로운 AI 스크린 기술을 공개했다 용석우 삼성전자 영상디스플레이사업부 사장은 올해 초 CES를 통해 AI 스크린 시대를 선언했다며 기존 시청 위주의 TV를 넘어 집 안의 다양한 기기를 연결하고 제어하는 AI 홈 디바이스로서 AI TV가 중심 역할을 수행할 것이라고 강조했다 용석우 삼성전자 영상디스플레이사업부 사장이 22일 삼성전자 수원사업장 디지털연구소에서 AI 스크린 미디어 브리핑을 하고 있다 용 사장은 올해 삼성전자 AI TV를 구매한 고객에게 앞으로 7년간 새로운 AI 기능이 탑재된 타이젠 OS 업그레이드를 무상으로 지원할 것이라고 밝혔다 삼성전자가 개발한 운영체제인 타이젠 OS는 삼성 TV에 적용된 세계 최대 스마트 TV 플랫폼 중 하나다 2023년 기준 2억 7000만대 이상의 삼성 스마트 TV에 적용돼 있다 삼성전자는 타이젠 OS 무상 업그레이드로 AI 홈 라이프 대중화를 앞당긴다는 계획이다삼성 AI TV는 집안에서 AI 홈 허브 역할을 수행하게 된다 별도의 허브 기기 없이도 집안의 AI 가전과 조명 커튼 플러그 도어록 등 집안 기기를 연결할 수 있다 3D 맵 뷰 기능을 활성화해 집 안의 평면도를 그대로 불러와 공간별 배치를 시각적으로 파악한다 따라서 한 눈에 집에 조명이 몇 개 켜있는지 알 수 있고 방마다 에어컨 전원을 끄고 켜는 등 기기를 관리할 수 있다 꺼져 있는 TV 앞에서 하이 빅스비를 부르면 오늘 날씨부터 에너지 사용량 우리 집 사물인터넷 기기 상태

In [ ]:
# 모델 평가모델

In [20]:
from sklearn.model_selection import train_test_split

# 데이터셋을 학습 데이터와 테스트 데이터로 분리
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# 학습 데이터로 학습
train_tagged_corpus = [TaggedDocument(tags=[row['title']], words=mecab.morphs(row['content'])) for index, row in train_df.iterrows()]
test_tagged_corpus = [TaggedDocument(tags=[row['title']], words=mecab.morphs(row['content'])) for index, row in test_df.iterrows()]

# Doc2Vec 모델 학습
model = doc2vec.Doc2Vec(vector_size=300, alpha=0.025, min_alpha=0.025, workers=8, window=8)
model.build_vocab(train_tagged_corpus)
model.train(train_tagged_corpus, total_examples=model.corpus_count, epochs=10)


In [21]:
from sklearn.metrics import accuracy_score

def evaluate_model(model, test_corpus):
    correct = 0
    total = len(test_corpus)

    for doc in test_corpus:
        # 테스트 데이터의 각 문서에 대해 유사한 문서 찾기
        inferred_vector = model.infer_vector(doc.words)
        most_similar = model.dv.most_similar([inferred_vector], topn=1)

        # 가장 유사한 문서의 제목이 원래의 제목과 같은지 확인
        if most_similar[0][0] == doc.tags[0]:
            correct += 1

    accuracy = correct / total
    print(f"Accuracy: {accuracy:.2%}")

# 모델 평가
evaluate_model(model, test_tagged_corpus)

Accuracy: 17.02%


In [23]:
# 예시: "삼성"이라는 단어와 가장 유사한 단어 찾기
similar_words = model.wv.most_similar("삼성", topn=5)
print("삼성:")
for word, similarity in similar_words:
    print(f"{word}: {similarity:.2f}")

삼성:
동양: 0.32
invest: 0.31
HJ: 0.29
가와사키: 0.29
삼성전자: 0.29
